# Conway's Algorithm in Python through the Martingale Strategy

## Imports

In [19]:
import math
import numpy as np
import pandas as pd
import itertools

## Functions

In [20]:
def create_cases(num_of_chars: int, length_of_case: int) -> list[np.ndarray]:
    """
    Function to create all possible case combinations for when the number of characters available is equal to num_of_chars and the length of the cases is equal to length_of_case.
    
    :param num_of_chars: The maximum number of distinct characters that can be used to create each case.
    :param length_of_case: The length of all cases
    :return: A list containing each combination in the form of numpy© arrays.
    """
    return [np.array(i) for i in itertools.product(range(num_of_chars), repeat=length_of_case)]

In [21]:
def gcd_euclid(a: float, b: float):
    while not b == 0:
        a, b = b, a % b
        
    return a

In [22]:
def conways_function(standard: np.ndarray, comparer: np.ndarray, number_of_characters: int, length_of_cases: int) -> int:
    """
    Function that uses the Martingale Strategy to compute the value of (number_of_characters)-base string comparisons used in Conway's Algorithm
    
    :param standard: The (number_of_characters)-base string that is compared to
    :param comparer: The (number_of_characters)-base string that is being compared
    :param number_of_characters: The maximum number of distinct characters that can be used in each case.
    :param length_of_cases: The length of all cases
    :return: The value of binary string comparison between the standard and comparer (number_of_characters)-base combination
    """
    result = 0
    for i in range(length_of_cases):
        test = comparer[i:]
        power = 0
        for j in range(length_of_cases - (i + 1), -1, -1):
            if test[j] == standard[j]:
                power += 1
            else:
                break
        
        if power != 0:
            result += math.pow(number_of_characters, power)
    
    # print(result, end="\t")
    return result

In [23]:
def relative_win_probability(standard: np.ndarray, comparer: np.ndarray, num_of_char: int, len_of_case: int) -> tuple[int, int]:
    """
    Calculates, according to Conway's Algorithm, the relative chance the player with the standard (num_of_char)-base string (player A) beats the player with the comparer (num_of_char)-base string (player B).
    
    :param standard: The (number_of_characters)-base string that is compared to.
    :param comparer: The (number_of_characters)-base string that is being compared.
    :param num_of_char: The maximum number of distinct characters that can be used in each case.
    :param len_of_case: The length of all cases.
    :return: The relative chance for player A to win over player B, or P(A win) / P(B win).
    """
    dividend = conways_function(comparer, comparer, num_of_char, len_of_case) - conways_function(comparer, standard, num_of_char, len_of_case)
    divisor = conways_function(standard, standard, num_of_char, len_of_case) - conways_function(standard, comparer, num_of_char, len_of_case)
    
    # print(dividend, "/", divisor)
    return dividend, divisor

In [24]:
def win_probability(number_of_characters: int, length_of_case: int) -> pd.DataFrame:
    """
    Calculates the win probability of player A's case over player B's case in the form of a data frame with each case for the indexes and columns
    
    :param number_of_characters: The maximum number of distinct characters that can be used in each case.
    :param length_of_case: The length of all cases.
    :return: A dataframe containing the probabilities to win 
    """
    cases = create_cases(num_of_chars=number_of_characters, length_of_case=length_of_case)
    indexes = []
    for case in cases:
        indexes.append(np.array2string(case))
    
    df = pd.DataFrame(columns=indexes, index=indexes)
    
    for case in cases:
        df.loc[np.array2string(case), np.array2string(case)] = 1.0
    
    for i in range(len(cases)):
        for j in range(i + 1, len(cases)):
            ratio = relative_win_probability(standard=cases[i], comparer=cases[j], num_of_char=number_of_characters, len_of_case=length_of_case)
            # minimizer = gcd_euclid(ratio[0], ratio[1])
            
            df.loc[np.array2string(cases[i]), np.array2string(cases[j])] = round(ratio[0] / (ratio[0] + ratio[1]) * 1000) / 1000
            # (ratio[0] // minimizer, (ratio[0] + ratio[1]) // minimizer)
            # ratio[0] / (ratio[0] + ratio[1])
            df.loc[np.array2string(cases[j]), np.array2string(cases[i])] = round(ratio[1] / (ratio[0] + ratio[1]) * 1000) / 1000
            # (ratio[1] // minimizer, (ratio[0] + ratio[1]) // minimizer)
            # ratio[1] / (ratio[0] + ratio[1])
            
    return df

In [25]:
def record_on_csv(probability_chart: pd.DataFrame, num_of_chars: int, len_of_cases: int) -> bool:
    try:
        probability_chart.to_csv(f'./conway_algorithm_results/win_prob_chart_{num_of_chars}_{len_of_cases}.csv')
        return True
    except FileNotFoundError:
        return False

## Execute / Main

In [26]:
def main():
    m = int(input("Input the number of characters to use: "))
    n = int(input("Input the length of case to use: "))
    win_probability_chart = win_probability(number_of_characters=m, length_of_case=n)
    
    # print("Win Probability Chart: ", end="\n\n")
    # print(win_probability_chart)
    
    record_on_csv(probability_chart=win_probability_chart, num_of_chars=m, len_of_cases=n)

In [27]:
if __name__ == '__main__':
    main()